In [1]:
import pandas as pd

In [117]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB

In [83]:
from collections import Counter

In [116]:
from imblearn.under_sampling import RandomUnderSampler

In [65]:
_2019 = pd.read_csv('2019_text_type.csv', index_col='Unnamed: 0')

In [97]:
_2019_df = _2019.copy()

In [98]:
X = _2019_df.text

In [99]:
X = (list(X))

In [100]:
y = _2019_df.type_of_material

In [101]:
# #funtion to cut off extra paragraphs from beautiful soup scrape

# def split_trim(string):
#     text = string.split(" ")
#     text = text[3:-14]
#     return text

In [102]:
# function to vectorize the type_of_material series into a y target vector.

def vectorize_type(ser):
    y = ser.copy()
    y.replace({'Op-Ed': 1,'News': 0}, inplace=True)
    return y

In [103]:
y = vectorize_type(y)

In [104]:
#X = X.apply(lambda x: split_trim(x))

# vectorize X first

In [105]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(X)

# work with imbalanced learn

In [109]:
y.shape

(41748,)

In [110]:
X.shape

(41748, 219112)

In [42]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)

In [113]:
X_resampled, y_resampled = rus.fit_resample(X, y)


#return a list of tuples for item, and count of item. in this case 4139 each
print(sorted(Counter(y_resampled).items()))

[(0, 4139), (1, 4139)]


In [114]:
y_resampled.shape

(8278,)

In [115]:
X_resampled.shape

(8278, 219112)

# Multinomial Naive Bayes/imbalanced learn

In [154]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=50)

In [155]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [156]:
clf.class_count_

array([3119., 3089.])

In [157]:
y_pred = clf.predict(X_test)

### clf.score = accuracy = 'true'(pos/neg) / total

In [163]:
#clf.score = accuracy = 'true'(pos/neg) / total

clf.score(X_test, y_test)

0.7840579710144927

In [159]:
confusion_matrix(y_true = y_test, y_pred = y_pred)

array([[ 590,  430],
       [  17, 1033]])

In [160]:
tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = y_pred).ravel()
(tn, fp, fn, tp)

(590, 430, 17, 1033)

In [162]:
(590 + 1033) / (430 + 17 + 590 + 1033)

0.7840579710144927